In [38]:
# TO BE REMOVED!!!
%load_ext autoreload
%autoreload 2
import os, shutil
import nglview as nv
import ipywidgets
import zipfile
import webbrowser

antibody_id = '4G6K'
antigen_id  = '4I1B'
complex_id  = '4C6M'
out_path = 'data/antibody/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
# Helpers
def def_dict(propierties={}):
    def_props = {'out_log_path': 'log/log.log',
                 'err_log_path': 'log/log.err',
                 'remove_tmp': True,
                 'can_write_console_log': True}
    def_props.update(propierties)
    return def_props

def show_pdbs(pdbs, surface=False):
    # Load the PDB files
    views = [nv.show_file(pdb) for pdb in pdbs]
    for view in views:
        if surface:
            view.clear()
            view.add_cartoon(color='black')
            view.add_surface(color='electrostatic', opacity=0.5)
        view.layout.width = '100%'
    return ipywidgets.HBox(views)

def display_actpass(pdb, actpass, opacity=1):
    with open(actpass, 'r') as file:
        actpass = file.read().splitlines()
        act_res = actpass[0].replace(' ', ', ')
        pas_res = actpass[1].replace(' ', ', ')
        
    # Load the PDB files
    view = nv.NGLWidget()
    view.add_component(pdb)
    view.clear()
    view.add_cartoon(color='black')
    view.add_ball_and_stick(color='grey',opacity=opacity)
    view.add_surface(selection=f'not ( {pas_res}, {act_res} )', color='white', opacity=opacity)
    if act_res != '':
        view.add_surface(selection=f'{act_res}', color='red')
    if pas_res != '':
        view.add_surface(selection=f'{pas_res}', color='green', opacity=opacity)
    view.layout.width = '100%'
    return view

# Antibody use case

***
This tutorial aims to illustrate the process of 
***


## Settings

### Biobb modules used

 - [biobb_io](https://github.com/bioexcel/biobb_io): Tools to fetch biomolecular data from public databases.
 - [biobb_model](https://github.com/bioexcel/biobb_model): Tools to model macromolecular structures.
 - [biobb_gromacs](https://github.com/bioexcel/biobb_gromacs): Tools to setup and run Molecular Dynamics simulations.
 - [biobb_analysis](https://github.com/bioexcel/biobb_analysis): Tools to analyse Molecular Dynamics trajectories.
 
### Auxiliar libraries used

 - [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels): Enables a Jupyter Notebook or JupyterLab application in one conda environment to access kernels for Python, R, and other languages found in other environments.
 - [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.
 - [ipywidgets](https://github.com/jupyter-widgets/ipywidgets): Interactive HTML widgets for Jupyter notebooks and the IPython kernel.
 - [plotly](https://plot.ly/python/offline/): Python interactive graphing library integrated in Jupyter notebooks.
 - [simpletraj](https://github.com/arose/simpletraj): Lightweight coordinate-only trajectory reader based on code from GROMACS, MDAnalysis and VMD.

### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_md_setup.git
cd biobb_wf_md_setup
conda env create -f conda_env/environment.yml
conda activate biobb_GMX_MDsetup_tutorial
jupyter nbextension enable python-markdown/main
jupyter-notebook biobb_wf_md_setup/notebooks/biobb_MDsetup_tutorial.ipynb
```

Please execute the following commands before launching the Jupyter Notebook if you experience some issues with widgets such as NGL View (3D molecular visualization):

```console
jupyter-nbextension enable --py --user widgetsnbextension
jupyter-nbextension enable --py --user nglview
```

***


## Pipeline steps


 1. [Input Parameters](#input)
 2. [Fetching PDB Structure](#fetch)
 3. [Fix Protein Structure](#fix)
 4. [Create Protein System Topology](#top)
 5. [Create Solvent Box](#box)
 6. [Fill the Box with Water Molecules](#water)
 7. [Adding Ions](#ions)
 8. [Energetically Minimize the System](#min)
 9. [Equilibrate the System (NVT)](#nvt)
 10. [Equilibrate the System (NPT)](#npt)
 11. [Free Molecular Dynamics Simulation](#free)
 12. [Post-processing and Visualizing Resulting 3D Trajectory](#post)
 13. [Output Files](#output)
 14. [Questions & Comments](#questions)
 
***
<img src="https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png" alt="Bioexcel2 logo"
    title="Bioexcel2 logo" width="400" />
***


## Sampling using ensemble from MD with GROMACS

In [ ]:
# https://mmb.irbbarcelona.org/biobb/workflows/tutorials/biobb_wf_flexdyn ?
# https://github.com/bioexcel/biobb_wf_flexdyn
# https://github.com/alevil-gmx/workflow_template/blob/main/workflow_template/workshop-template-ab.ipynb

<a id="input"></a>
### Input parameters
**Input parameters** needed:
 - **pdbCode**: PDB code of the protein structure (e.g. 1AKI)

In [40]:
cwd = os.getcwd()
MD_dir = out_path+'1_MD/'
MD_inp = MD_dir+'input/'
os.makedirs(MD_inp, exist_ok=True) 
pdbCode = "4G6K"

<a id="fetch"></a>
***
### Fetching PDB structure
Downloading **PDB structure** with the **protein molecule** from the RCSB PDB database.<br>
Alternatively, a **PDB file** can be used as starting structure. <br>
***
**Building Blocks** used:
 - [Pdb](https://biobb-io.readthedocs.io/en/latest/api.html#module-api.pdb) from **biobb_io.api.pdb**
***

In [6]:
# Downloading desired PDB file 
# Import module
from biobb_io.api.pdb import pdb

# Create properties dict and inputs/outputs
downloaded_pdb = f'{MD_dir}{pdbCode}.pdb'
prop = {
    'pdb_code': pdbCode
}

# Create and launch bb
pdb(output_pdb_path=downloaded_pdb,
    properties=def_dict(prop))

2025-06-04 10:22:57,873 [MainThread  ] [INFO ]  Module: biobb_io.api.pdb Version: 5.0.1
2025-06-04 10:22:57,873 [MainThread  ] [INFO ]  Downloading 4g6k from: https://www.ebi.ac.uk/pdbe/entry-files/download/pdb4g6k.ent
2025-06-04 10:22:58,182 [MainThread  ] [INFO ]  Writting pdb to: data/antibody/1_MD/4G6K.pdb
2025-06-04 10:22:58,182 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'MODEL', 'ENDMDL']
2025-06-04 10:22:58,186 [MainThread  ] [INFO ]  


0

<a id="vis3D"></a>
#### Visualizing 3D structure
Visualizing the downloaded/given **PDB structure** using **NGL**:    

In [ ]:
# Show protein
view = nv.show_structure_file(downloaded_pdb)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

<a id="fix"></a>
***
### Fix protein structure
**Checking** and **fixing** (if needed) the protein structure:<br>
- **Modeling** **missing side-chain atoms**, modifying incorrect **amide assignments**, choosing **alternative locations**.<br>
- **Checking** for missing **backbone atoms**, **heteroatoms**, **modified residues** and possible **atomic clashes**.

***
**Building Blocks** used:
 - [FixSideChain](https://biobb-model.readthedocs.io/en/latest/model.html#module-model.fix_side_chain) from **biobb_model.model.fix_side_chain**
***

In [181]:
# Check & Fix PDB
# Import module
from biobb_model.model.fix_side_chain import fix_side_chain

# Create prop dict and inputs/outputs
fixed_pdb = f'{MD_dir}{pdbCode}_fixed.pdb'

# Create and launch bb
fix_side_chain(input_pdb_path=downloaded_pdb, 
               output_pdb_path=fixed_pdb,
               properties=def_dict())

2025-06-04 15:11:28,210 [MainThread  ] [INFO ]  Module: biobb_model.model.fix_side_chain Version: 5.0.0
2025-06-04 15:11:28,211 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_46bb2f6b-973c-45f8-85cc-935e5bc9b9a6 directory successfully created
2025-06-04 15:11:28,211 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K.pdb to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_46bb2f6b-973c-45f8-85cc-935e5bc9b9a6
2025-06-04 15:11:28,212 [MainThread  ] [INFO ]  check_structure -i /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_46bb2f6b-973c-45f8-85cc-935e5bc9b9a6/4G6K.pdb -o /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_46bb2f6b-973c-45f8-85cc-935e5bc9b9a6/4G6K_fixed.pdb --force_save fixside --fix ALL

2025-06-04 15:11:28,435 [MainThread  ] [INFO ]  Executing: check_structure -i /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/...

0

#### Visualizing 3D structure
Visualizing the fixed **PDB structure** using **NGL**. In this particular example, the checking step didn't find any issue to be solved, so there is no difference between the original structure and the fixed one.   

In [8]:
# Show protein
view = nv.show_structure_file(fixed_pdb)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<a id="top"></a>
***
### Create protein system topology
**Building GROMACS topology** corresponding to the protein structure.<br>
Force field used in this tutorial is [**amber99sb-ildn**](https://dx.doi.org/10.1002%2Fprot.22711): AMBER **parm99** force field with **corrections on backbone** (sb) and **side-chain torsion potentials** (ildn). Water molecules type used in this tutorial is [**spc/e**](https://pubs.acs.org/doi/abs/10.1021/j100308a038).<br>
Adding **hydrogen atoms** if missing. Automatically identifying **disulfide bridges**. <br>

Generating two output files: 
- **GROMACS structure** (gro file)
- **GROMACS topology** ZIP compressed file containing:
    - *GROMACS topology top file* (top file)
    - *GROMACS position restraint file/s* (itp file/s)
***
**Building Blocks** used:
 - [Pdb2gmx](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.pdb2gmx) from **biobb_gromacs.gromacs.pdb2gmx**
***

In [9]:
# Downloading CHARMM36 FF
!curl  -o {MD_inp}charmm36-jul2017.ff.tgz https://mackerell.umaryland.edu/download.php?filename=CHARMM_ff_params_files/charmm36-jul2017.ff.tgz
# Unzip CHARMM36 FF
!tar -xvzf {MD_inp}charmm36-jul2017.ff.tgz -C {MD_inp}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  586k  100  586k    0     0   162k      0  0:00:03  0:00:03 --:--:--  162k
charmm36-jul2017.ff/
charmm36-jul2017.ff/spc.itp
charmm36-jul2017.ff/ffbonded.itp
charmm36-jul2017.ff/merged.rtp
charmm36-jul2017.ff/nbfix.itp
charmm36-jul2017.ff/gb.itp
charmm36-jul2017.ff/forcefield.doc
charmm36-jul2017.ff/merged.hdb
charmm36-jul2017.ff/ions.itp
charmm36-jul2017.ff/merged.arn
charmm36-jul2017.ff/merged.vsd
charmm36-jul2017.ff/spce.itp
charmm36-jul2017.ff/old_c36_cmap.itp
charmm36-jul2017.ff/ffnonbonded.itp
charmm36-jul2017.ff/atomtypes.atp
charmm36-jul2017.ff/merged.c.tdb
charmm36-jul2017.ff/forcefield.itp
charmm36-jul2017.ff/watermodels.dat
charmm36-jul2017.ff/tip4p.itp
charmm36-jul2017.ff/cmap.itp
charmm36-jul2017.ff/merged.n.tdb
charmm36-jul2017.ff/tip3p.itp
charmm36-jul2017.ff/merged.r2b


In [10]:
# Create system topology
# Import module
from biobb_gromacs.gromacs.pdb2gmx import pdb2gmx

# Create inputs/outputs
output_pdb2gmx_gro     = f'{MD_dir}{pdbCode}__pdb2gmx.gro'
output_pdb2gmx_top_zip = f'{MD_dir}{pdbCode}__pdb2gmx_top.zip'

prop = {
    'force_field' : 'charmm36-jul2017',
    'gmx_lib' : f'{cwd}/{MD_inp}',
    'water_type' : 'tip3p',
    #'force_field' : 'amber99sb-ildn'
}

# Create and launch bb
pdb2gmx(input_pdb_path=fixed_pdb, 
        output_gro_path=output_pdb2gmx_gro, 
        output_top_zip_path=output_pdb2gmx_top_zip,
        properties=def_dict(prop))

2025-06-04 10:23:10,718 [MainThread  ] [INFO ]  Module: biobb_gromacs.gromacs.pdb2gmx Version: 5.0.0
2025-06-04 10:23:10,719 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_789d6e3f-9f0c-4f99-bbea-bf3693f7b5b1 directory successfully created
2025-06-04 10:23:10,719 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K_fixed.pdb to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_789d6e3f-9f0c-4f99-bbea-bf3693f7b5b1
2025-06-04 10:23:10,720 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright pdb2gmx -f /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_789d6e3f-9f0c-4f99-bbea-bf3693f7b5b1/4G6K_fixed.pdb -o /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_789d6e3f-9f0c-4f99-bbea-bf3693f7b5b1/4G6K__pdb2gmx.gro -p p2g.top -water tip3p -ff charmm36-jul2017 -i posre.itp

2025-06-04 10:23:11,336 [MainThread  ] [INFO ]  Executing: gmx -nobackup -nocopyrigh

0

#### Visualizing 3D structure
Visualizing the generated **GRO structure** using **NGL**. Note that **hydrogen atoms** were added to the structure by the **pdb2gmx GROMACS tool** when generating the **topology**.    

In [11]:
# Show protein
view = nv.show_structure_file(output_pdb2gmx_gro)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<a id="box"></a>
***
### Create solvent box
Define the unit cell for the **protein structure MD system** to fill it with water molecules.<br>
A **cubic box** is used to define the unit cell, with a **distance from the protein to the box edge of 1.0 nm**. The protein is **centered in the box**.  

***
**Building Blocks** used:
 - [Editconf](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.editconf) from **biobb_gromacs.gromacs.editconf** 
***

In [12]:
# Editconf: Create solvent box
# Import module
from biobb_gromacs.gromacs.editconf import editconf

# Create prop dict and inputs/outputs
output_editconf_gro = f'{MD_dir}{pdbCode}_editconf.gro'

prop = {
    'box_type': 'dodecahedron',
    'distance_to_molecule': 0.7
}

#Create and launch bb
editconf(input_gro_path=output_pdb2gmx_gro, 
         output_gro_path=output_editconf_gro,
         properties=def_dict(prop))

2025-06-04 10:23:11,490 [MainThread  ] [INFO ]  Module: biobb_gromacs.gromacs.editconf Version: 5.0.0
2025-06-04 10:23:11,491 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_9102d68a-bbfd-4693-aade-279df55d9e4a directory successfully created
2025-06-04 10:23:11,492 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K__pdb2gmx.gro to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_9102d68a-bbfd-4693-aade-279df55d9e4a
2025-06-04 10:23:11,493 [MainThread  ] [INFO ]  Distance of the box to molecule:   0.70
2025-06-04 10:23:11,493 [MainThread  ] [INFO ]  Centering molecule in the box.
2025-06-04 10:23:11,494 [MainThread  ] [INFO ]  Box type: dodecahedron
2025-06-04 10:23:11,494 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright editconf -f /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_9102d68a-bbfd-4693-aade-279df55d9e4a/4G6K__pdb2gmx.gro -o /home/rchaves/repo/biobb/biobb_

2025-06-04 10:23:11,529 [MainThread  ] [INFO ]  Executing: gmx -nobackup -nocopyright editconf -f /home/rchaves/repo/biobb/biobb_wf_antibod...
2025-06-04 10:23:11,530 [MainThread  ] [INFO ]  Exit code: 0
2025-06-04 10:23:11,530 [MainThread  ] [INFO ]  Note that major changes are planned in future for editconf, to improve usability and utility.
Read 6525 atoms
Volume: 244.744 nm^3, corresponds to roughly 110100 electrons
No velocities found
    system size :  6.427  5.139  7.410 (nm)
    diameter    :  7.999               (nm)
    center      :  4.070  2.468 -2.706 (nm)
    box vectors :  6.427  5.139  7.410 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  : 244.74               (nm^3)
    shift       :  2.979  4.582  6.029 (nm)
new center      :  7.049  7.049  3.323 (nm)
new box vectors :  9.399  9.399  9.399 (nm)
new box angles  :  60.00  60.00  90.00 (degrees)
new box volume  : 587.15               (nm^3)

2025-06-04 10:23:11,530 [MainThread  ] [INFO ]           

0

<a id="water"></a>
***
### Fill the box with water molecules
Fill the unit cell for the **protein structure system** with water molecules.<br>
The solvent type used is the default **Simple Point Charge water (SPC)**, a generic equilibrated 3-point solvent model. 

***
**Building Blocks** used:
 - [Solvate](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.solvate) from **biobb_gromacs.gromacs.solvate** 
***

In [13]:
# Solvate: Fill the box with water molecules
from biobb_gromacs.gromacs.solvate import solvate

# Create prop dict and inputs/outputs
output_solvate_gro     = f'{MD_dir}{pdbCode}_solvate.gro'
output_solvate_top_zip = f'{MD_dir}{pdbCode}_solvate_top.zip'

# Create and launch bb
solvate(input_solute_gro_path=output_editconf_gro, 
        output_gro_path=output_solvate_gro, 
        input_top_zip_path=output_pdb2gmx_top_zip, 
        output_top_zip_path=output_solvate_top_zip,
        properties=def_dict(prop))

2025-06-04 10:23:11,709 [MainThread  ] [INFO ]  Module: biobb_gromacs.gromacs.solvate Version: 5.0.0
2025-06-04 10:23:11,709 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_972867ae-b497-4c79-9e0a-10756ac7fe42 directory successfully created
2025-06-04 10:23:11,710 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K_editconf.gro to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_972867ae-b497-4c79-9e0a-10756ac7fe42
2025-06-04 10:23:11,713 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/data/antibody/1_MD/4G6K__pdb2gmx_top.zip
2025-06-04 10:23:11,713 [MainThread  ] [INFO ]  to:
2025-06-04 10:23:11,714 [MainThread  ] [INFO ]  ['466c834d-3108-41fd-8c85-d2196d062d0e/p2g.top', '466c834d-3108-41fd-8c85-d2196d062d0e/p2g_Protein_chain_H.itp', '466c834d-3108-41fd-8c85-d2196d062d0e/p2g_Protein_chain_L.itp', '466c834d-3108-41fd-8c85-d2196d062d0e/posre_P

/home/rchaves/miniforge3/envs/biobb_wf_antibody/lib/python3.10/site-packages/biobb_common/generic/biobb_object.py:183: UserWarning: Warning: distance_to_molecule is not a recognized property. The most similar property is: stage_io_dict
  warnings.warn(
/home/rchaves/miniforge3/envs/biobb_wf_antibody/lib/python3.10/site-packages/biobb_common/generic/biobb_object.py:183: UserWarning: Warning: box_type is not a recognized property. The most similar property is: sandbox_path
  warnings.warn(


2025-06-04 10:23:11,941 [MainThread  ] [INFO ]  Executing: gmx -nobackup -nocopyright solvate -cp /home/rchaves/repo/biobb/biobb_wf_antibod...
2025-06-04 10:23:11,942 [MainThread  ] [INFO ]  Exit code: 0
2025-06-04 10:23:11,942 [MainThread  ] [INFO ]  
         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.

NOTE: From version 5.0 gmx solvate uses the Van der Waals radii
from the source below. This means the results may be different
compared to previous GROMACS versions.

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
A. Bondi
van der Waals Volumes and Radii
J. Phys. Chem. 68 (1964) pp. 441-451
------

0

#### Visualizing 3D structure
Visualizing the **protein system** with the newly added **solvent box** using **NGL**.<br> Note the **cubic box** filled with **water molecules** surrounding the **protein structure**, which is **centered** right in the middle of the cube.

In [14]:
# Show protein
view = nv.show_structure_file(output_solvate_gro)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='solute', color='green')
view.add_representation(repr_type='ball+stick', selection='SOL')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<a id="ions"></a>
***
### Adding ions
Add ions to neutralize the **protein structure** charge
- [Step 1](#ionsStep1): Creating portable binary run file for ion generation
- [Step 2](#ionsStep2): Adding ions to **neutralize** the system
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_gromacs.gromacs.grompp** 
 - [Genion](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.genion) from **biobb_gromacs.gromacs.genion** 
***

<a id="ionsStep1"></a>
#### Step 1: Creating portable binary run file for ion generation
A simple **energy minimization** molecular dynamics parameters (mdp) properties will be used to generate the portable binary run file for **ion generation**, although **any legitimate combination of parameters** could be used in this step.

In [15]:
# Grompp: Creating portable binary run file for ion generation
from biobb_gromacs.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppion_tpr = f'{MD_dir}{pdbCode}_gppion.tpr'
prop = {
    'simulation_type': 'ions',
    'gmx_lib' : f'{cwd}/{MD_inp}',
    'maxwarn': 1
}

# Create and launch bb
grompp(input_gro_path=output_solvate_gro, 
       input_top_zip_path=output_solvate_top_zip, 
       output_tpr_path=output_gppion_tpr,  
       properties=def_dict(prop))

2025-06-04 10:23:15,595 [MainThread  ] [INFO ]  Module: biobb_gromacs.gromacs.grompp Version: 5.0.0
2025-06-04 10:23:15,596 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_4c55a9db-0a19-44d6-81b9-c92d24a76d4b directory successfully created
2025-06-04 10:23:15,598 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K_solvate.gro to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_4c55a9db-0a19-44d6-81b9-c92d24a76d4b
2025-06-04 10:23:15,601 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/data/antibody/1_MD/4G6K_solvate_top.zip
2025-06-04 10:23:15,601 [MainThread  ] [INFO ]  to:
2025-06-04 10:23:15,601 [MainThread  ] [INFO ]  ['/home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_4c55a9db-0a19-44d6-81b9-c92d24a76d4b/p2g.top', '/home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_4c55a9db-0a19-44d

0

<a id="ionsStep2"></a>
#### Step 2: Adding ions to neutralize the system
Replace **solvent molecules** with **ions** to **neutralize** the system.

In [16]:
# Genion: Adding ions to neutralize the system
from biobb_gromacs.gromacs.genion import genion

# Create prop dict and inputs/outputs
output_genion_gro     = f'{MD_dir}{pdbCode}_genion.gro'
output_genion_top_zip = f'{MD_dir}{pdbCode}_genion_top.zip'
prop={
    'neutral':True,
    'ionic_concentration' : 0.15
}

# Create and launch bb
genion(input_tpr_path=output_gppion_tpr, 
       output_gro_path=output_genion_gro, 
       input_top_zip_path=output_solvate_top_zip, 
       output_top_zip_path=output_genion_top_zip, 
       properties=def_dict(prop))

2025-06-04 10:23:16,480 [MainThread  ] [INFO ]  Module: biobb_gromacs.gromacs.genion Version: 5.0.0
2025-06-04 10:23:16,481 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_587be689-12d0-4dfb-86fc-1871ebfe911c directory successfully created
2025-06-04 10:23:16,484 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K_gppion.tpr to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_587be689-12d0-4dfb-86fc-1871ebfe911c
2025-06-04 10:23:16,484 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/0a322edb-9f2a-48b8-ad8a-b7504ab4cbe6.stdin to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_587be689-12d0-4dfb-86fc-1871ebfe911c
2025-06-04 10:23:16,489 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/data/antibody/1_MD/4G6K_solvate_top.zip
2025-06-04 10:23:16,489 [MainThread  ] 

/home/rchaves/miniforge3/envs/biobb_wf_antibody/lib/python3.10/site-packages/biobb_common/generic/biobb_object.py:183: UserWarning: Warning: ionic_concentration is not a recognized property. The most similar property is: concentration
  warnings.warn(


0

#### Visualizing 3D structure
Visualizing the **neutralized protein system** with the newly added **ions** using **NGL**

In [17]:
# Show protein
view = nv.show_structure_file(output_genion_gro)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='solute', color='sstruc')
view.add_representation(repr_type='ball+stick', selection='NA')
view.add_representation(repr_type='ball+stick', selection='CL')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<a id="min"></a>
***
### Energetically minimize the system
Energetically minimize the **protein system** till reaching a desired potential energy.
- [Step 1](#emStep1): Creating portable binary run file for energy minimization
- [Step 2](#emStep2): Energetically minimize the **system** till reaching a force of 500 kJ mol-1 nm-1.
- [Step 3](#emStep3): Checking **energy minimization** results. Plotting energy by time during the **minimization** process.
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_gromacs.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_gromacs.gromacs.mdrun** 
 - [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy** 
***

<a id="emStep1"></a>
#### Step 1: Creating portable binary run file for energy minimization
The **minimization** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **energy minimization**:

-  integrator  = steep ; Algorithm (steep = steepest descent minimization)
-  emtol       = 1000.0 ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
-  emstep      = 0.01 ; Minimization step size (nm)
-  nsteps      = 50000 ; Maximum number of (minimization) steps to perform

In this particular example, the method used to run the **energy minimization** is the default **steepest descent**, but the **maximum force** is placed at **500 KJ/mol\*nm^2**, and the **maximum number of steps** to perform (if the maximum force is not reached) to **5,000 steps**. 

In [18]:
# https://github.com/alevil-gmx/workflow_template/blob/main/workflow_template/data/input/emin-charmm.mdp
input_mdp_min = f"{MD_inp}emin-charmm.mdp"
fl = """title       = CHARMM steepest descent enrgy minimisation

; Parameters describing what to do, when to stop and what to save
integrator  = steep  ; Algorithm (steep = steepest descent minimization)
emtol       = 1000.0 ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep      = 0.01   ; Minimization step size
nstenergy   = 500    ; save energies every 1.0 ps, so we can observe if we are successful
nsteps      = -1     ; run as long as we need
; Settings that make sure we run with parameters in harmony with the selected force-field
constraints             = h-bonds   ; bonds involving H are constrained
rcoulomb                = 1.2       ; short-range electrostatic cutoff (in nm)
rvdw                    = 1.2       ; short-range van der Waals cutoff (in nm)
vdw-modifier            = Force-switch ;  specific CHARMM
rvdw_switch             = 1.0       ;
DispCorr                = EnerPres  ; account for cut-off vdW scheme
coulombtype             = PME       ; Particle Mesh Ewald for long-range electrostatics
fourierspacing          = 0.15     ; grid spacing for FFT
"""

with open(input_mdp_min, 'w') as f:
    f.write(fl)

In [19]:
# Grompp: Creating portable binary run file for mdrun
from biobb_gromacs.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppmin_tpr = f'{MD_dir}{pdbCode}_gppmin.tpr'

prop = {
    'gmx_lib' : f'{cwd}/{MD_inp}',
}

# Create and launch bb
grompp(input_gro_path=output_genion_gro, 
       input_top_zip_path=output_genion_top_zip, 
       input_mdp_path=input_mdp_min,
       output_tpr_path=output_gppmin_tpr,  
       properties=def_dict(prop))

2025-06-04 10:23:27,211 [MainThread  ] [INFO ]  Module: biobb_gromacs.gromacs.grompp Version: 5.0.0
2025-06-04 10:23:27,211 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_c1122525-196a-42a6-ab36-ae95d333e980 directory successfully created
2025-06-04 10:23:27,213 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K_genion.gro to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_c1122525-196a-42a6-ab36-ae95d333e980
2025-06-04 10:23:27,213 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/input/emin-charmm.mdp to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_c1122525-196a-42a6-ab36-ae95d333e980
2025-06-04 10:23:27,216 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/data/antibody/1_MD/4G6K_genion_top.zip
2025-06-04 10:23:27,217 [MainThread  ] [INFO ]  to:
2025-06-04 10:23:27,217 [MainThread  ] [INFO ]  ['/home/rchav

0

<a id="emStep2"></a>
#### Step 2: Running Energy Minimization
Running **energy minimization** using the **tpr file** generated in the previous step. 

In [ ]:
# Mdrun: Running minimization
from biobb_gromacs.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_min_trr = f'{MD_dir}{pdbCode}_min.trr'
output_min_gro = f'{MD_dir}{pdbCode}_min.gro'
output_min_edr = f'{MD_dir}{pdbCode}_min.edr'
output_min_log = f'{MD_dir}{pdbCode}_min.log'

# Create and launch bb
mdrun(input_tpr_path=output_gppmin_tpr, 
      output_trr_path=output_min_trr, 
      output_gro_path=output_min_gro, 
      output_edr_path=output_min_edr, 
      output_log_path=output_min_log,
      properties=def_dict(prop))

<a id="emStep3"></a>
#### Step 3: Checking Energy Minimization results
Checking **energy minimization** results. Plotting **potential energy** by time during the minimization process. 

In [21]:
# GMXEnergy: Getting system energy by time  
from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_min_ene_xvg = f'{MD_dir}{pdbCode}_min_ene.xvg'
prop = {
    'terms':  ["Potential"]
}

# Create and launch bb
gmx_energy(input_energy_path=output_min_edr, 
          output_xvg_path=output_min_ene_xvg, 
          properties=def_dict(prop))

2025-06-04 10:23:48,123 [MainThread  ] [INFO ]  Module: biobb_analysis.gromacs.gmx_energy Version: 5.0.1
2025-06-04 10:23:48,123 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_d42ae4be-9ef7-473a-9e55-5532745d80f4 directory successfully created
2025-06-04 10:23:48,124 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/data/antibody/1_MD/4G6K_min.edr to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_d42ae4be-9ef7-473a-9e55-5532745d80f4
2025-06-04 10:23:48,125 [MainThread  ] [INFO ]  gmx energy -f /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_d42ae4be-9ef7-473a-9e55-5532745d80f4/4G6K_min.edr -o /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_d42ae4be-9ef7-473a-9e55-5532745d80f4/4G6K_min_ene.xvg -xvg none < 115e9708-f0c2-4866-bfcb-e774cdfe1b65/instructions.in

2025-06-04 10:23:48,140 [MainTh

0

In [22]:
import plotly
import plotly.graph_objs as go

#Read data from file and filter energy values higher than 1000 Kj/mol^-1
with open(output_min_ene_xvg,'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Energy Minimization",
                        xaxis=dict(title = "Energy Minimization Step"),
                        yaxis=dict(title = "Potential Energy KJ/mol-1")
                       )
}

plotly.offline.iplot(fig)

<a id="npt"></a>
***
### Equilibrate the system (NPT)
Equilibrate the **protein system** in **NPT** ensemble (constant Number of particles, Pressure and Temperature).
- [Step 1](#eqNPTStep1): Creating portable binary run file for system equilibration
- [Step 2](#eqNPTStep2): Equilibrate the **protein system** with **NPT** ensemble.
- [Step 3](#eqNPTStep3): Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process.
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_gromacs.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_gromacs.gromacs.mdrun** 
 - [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy** 
***

<a id="eqNPTStep1"></a>
#### Step 1: Creating portable binary run file for system equilibration (NPT)

The **npt** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **NPT equilibration** with **protein restraints** (see [GROMACS mdp options](http://manual.gromacs.org/documentation/2018/user-guide/mdp-options.html)):

-  Define                   = -DPOSRES
-  integrator               = md
-  dt                       = 0.002
-  nsteps                   = 5000
-  pcoupl = Parrinello-Rahman
-  pcoupltype = isotropic
-  tau_p = 1.0
-  ref_p = 1.0
-  compressibility = 4.5e-5
-  refcoord_scaling = com
-  gen_vel = no

In this particular example, the default parameters will be used: **md** integrator algorithm, a **time step** of **2fs**, **5,000 equilibration steps** with the protein **heavy atoms restrained**, and a Parrinello-Rahman **pressure coupling** algorithm.

*Please note that for the sake of time this tutorial is only running 10ps of NPT equilibration, whereas in the [original example](http://www.mdtutorials.com/gmx/lysozyme/07_equil2.html) the simulated time was 100ps.*

In [23]:
# From https://github.com/alevil-gmx/workflow_template/blob/main/workflow_template/data/input/md_eq_posre_charmm36m.mdp
# !! se cambia define -> Define y ns_type -> ns-type o el biobb da error (TO-DO arreglar)

input_mdp_eq = f"{MD_inp}md_eq_posre_charmm36m.mdp"
fl="""Define                   = -DPOSRES
integrator               = md
tinit                    = 0
dt                       = 0.002
nsteps                   = 500000   ; 1  ns
comm_grps                = system
nstxout                  = 0
nstvout                  = 0
nstfout                  = 0
nstlog                   = 10000
nstenergy                = 10000
nstxout-compressed       = 10000
compressed-x-grps        = system 
nstlist                  = 10
ns-type                  = grid
pbc                      = xyz
; 
coulombtype              = PME
rcoulomb                 = 1.2
fourierspacing           = 0.15 
;
vdwtype                  = Cut-off
vdw_modifier=Force-switch
rvdw_switch              = 1.0
rvdw                     = 1.2
DispCorr                 = EnerPres
;
constraints              = h-bonds
constraint_algorithm     = LINCS
; new
Pcoupl                   = Berendsen
tau_p                    = 5.0  
ref_p                    = 1.0 
compressibility          = 4.5e-5 
refcoord-scaling         = all     ; to use with pos restrain
Tcoupl                   = v-rescale 
tc-grps                  = system
tau_t                    = 0.5 
ref_t                    = 298 """
with open(input_mdp_eq, 'w') as f:
    f.write(fl)

In [24]:
# Grompp: Creating portable binary run file for NPT Equilibration
from biobb_gromacs.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppnpt_tpr = f'{MD_dir}{pdbCode}_gppnpt.tpr'

prop = {
    'mdp': {
        'nsteps' : 5000, # subir
    },
    'gmx_lib' : f'{cwd}/{MD_inp}',
    'maxwarn' : 1,
}

# Create and launch bb
grompp(input_gro_path=output_min_gro, 
       input_top_zip_path=output_genion_top_zip, 
       input_mdp_path=input_mdp_eq,
       output_tpr_path=output_gppnpt_tpr,  
       properties=def_dict(prop))

2025-06-04 10:23:48,451 [MainThread  ] [INFO ]  Module: biobb_gromacs.gromacs.grompp Version: 5.0.0
2025-06-04 10:23:48,452 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_a1841467-8a9d-47f8-8d5a-62eefb6c1cbc directory successfully created
2025-06-04 10:23:48,455 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K_min.gro to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_a1841467-8a9d-47f8-8d5a-62eefb6c1cbc
2025-06-04 10:23:48,455 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/input/md_eq_posre_charmm36m.mdp to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_a1841467-8a9d-47f8-8d5a-62eefb6c1cbc
2025-06-04 10:23:48,458 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/data/antibody/1_MD/4G6K_genion_top.zip
2025-06-04 10:23:48,459 [MainThread  ] [INFO ]  to:
2025-06-04 10:23:48,459 [MainThread  ] [INFO ]  ['/hom

0

<a id="eqNPTStep2"></a>
#### Step 2: Running NPT equilibration

In [25]:
# Mdrun: Running NPT System Equilibration
from biobb_gromacs.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_npt_trr = f'{MD_dir}{pdbCode}_npt.trr'
output_npt_gro = f'{MD_dir}{pdbCode}_npt.gro'
output_npt_edr = f'{MD_dir}{pdbCode}_npt.edr'
output_npt_log = f'{MD_dir}{pdbCode}_npt.log'
output_npt_cpt = f'{MD_dir}{pdbCode}_npt.cpt'

# Create and launch bb
mdrun(input_tpr_path=output_gppnpt_tpr, 
      output_trr_path=output_npt_trr, 
      output_gro_path=output_npt_gro, 
      output_edr_path=output_npt_edr, 
      output_log_path=output_npt_log, 
      output_cpt_path=output_npt_cpt,
      properties=def_dict())

2025-06-04 10:23:49,346 [MainThread  ] [INFO ]  Module: biobb_gromacs.gromacs.mdrun Version: 5.0.0
2025-06-04 10:23:49,346 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_b8d1f1ca-4b65-4973-a634-557a25edfc8f directory successfully created
2025-06-04 10:23:49,349 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K_gppnpt.tpr to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_b8d1f1ca-4b65-4973-a634-557a25edfc8f
2025-06-04 10:23:49,350 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -o /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_b8d1f1ca-4b65-4973-a634-557a25edfc8f/4G6K_npt.trr -s /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_b8d1f1ca-4b65-4973-a634-557a25edfc8f/4G6K_gppnpt.tpr -c /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_b8d1f1ca-4b65-4973-a634-557a25edfc8f/4G6K_npt.gro -e /home/rchaves/r

/home/rchaves/miniforge3/envs/biobb_wf_antibody/lib/python3.10/site-packages/biobb_common/tools/file_utils.py:770: UserWarning:

Path data/antibody/1_MD/4G6K_npt.trr --- biobb_gromacs.gromacs.mdrun: Unexisting output_trr_path file.



0

<a id="eqNPTStep3"></a>
#### Step 3: Checking NPT Equilibration results
Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process. 

In [ ]:
# GMXEnergy: Getting system pressure and density by time during NPT Equilibration  
from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_npt_pd_xvg = f'{MD_dir}{pdbCode}_npt_PD.xvg'
prop = {
    'terms':  ["Pressure","Density"]
}

# Create and launch bb
gmx_energy(input_energy_path=output_npt_edr, 
          output_xvg_path=output_npt_pd_xvg, 
          properties=def_dict(prop))

2025-06-04 10:25:24,031 [MainThread  ] [INFO ]  Module: biobb_analysis.gromacs.gmx_energy Version: 5.0.1
2025-06-04 10:25:24,032 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_c7bacdd8-1041-4125-8185-9e711ac9c30e directory successfully created
2025-06-04 10:25:24,032 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/data/antibody/1_MD/4G6K_npt.edr to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_c7bacdd8-1041-4125-8185-9e711ac9c30e
2025-06-04 10:25:24,033 [MainThread  ] [INFO ]  gmx energy -f /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_c7bacdd8-1041-4125-8185-9e711ac9c30e/4G6K_npt.edr -o /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_c7bacdd8-1041-4125-8185-9e711ac9c30e/4G6K_npt_PD.xvg -xvg none < c034ab8b-6250-4c26-90c9-a30512cabadd/instructions.in

2025-06-04 10:25:24,047 [MainThr

0

In [27]:
import plotly
from plotly import subplots
import plotly.graph_objs as go

# Read pressure and density data from file 
with open(output_npt_pd_xvg,'r') as pd_file:
    x,y,z = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]),float(line.split()[2]))
            for line in pd_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

trace1 = go.Scatter(
    x=x,y=y
)
trace2 = go.Scatter(
    x=x,y=z
)

fig = subplots.make_subplots(rows=1, cols=2, print_grid=False)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig['layout']['xaxis1'].update(title='Time (ps)')
fig['layout']['xaxis2'].update(title='Time (ps)')
fig['layout']['yaxis1'].update(title='Pressure (bar)')
fig['layout']['yaxis2'].update(title='Density (Kg*m^-3)')

fig['layout'].update(title='Pressure and Density during NPT Equilibration')
fig['layout'].update(showlegend=False)

plotly.offline.iplot(fig)

<a id="free"></a>
***
### Free Molecular Dynamics Simulation
Upon completion of the **two equilibration phases (NVT and NPT)**, the system is now well-equilibrated at the desired temperature and pressure. The **position restraints** can now be released. The last step of the **protein** MD setup is a short, **free MD simulation**, to ensure the robustness of the system. 
- [Step 1](#mdStep1): Creating portable binary run file to run a **free MD simulation**.
- [Step 2](#mdStep2): Run short MD simulation of the **protein system**.
- [Step 3](#mdStep3): Checking results for the final step of the setup process, the **free MD run**. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time during the **free MD run** step. 
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_gromacs.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_gromacs.gromacs.mdrun** 
 - [GMXRms](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_rms) from **biobb_analysis.gromacs.gmx_rms** 
 - [GMXRgyr](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_rgyr) from **biobb_analysis.gromacs.gmx_rgyr** 
***

<a id="mdStep1"></a>
#### Step 1: Creating portable binary run file to run a free MD simulation

The **free** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **free MD simulation** (see [GROMACS mdp options](http://manual.gromacs.org/documentation/2018/user-guide/mdp-options.html)):

-  integrator               = md
-  dt                       = 0.002 (ps)
-  nsteps                   = 50000

In this particular example, the default parameters will be used: **md** integrator algorithm, a **time step** of **2fs**, and a total of **50,000 md steps** (100ps).

*Please note that for the sake of time this tutorial is only running 100ps of free MD, whereas in the [original example](http://www.mdtutorials.com/gmx/lysozyme/08_MD.html) the simulated time was 1ns (1000ps).*

In [77]:
# https://github.com/alevil-gmx/workflow_template/blob/main/workflow_template/data/input/md_charmm36m.mdp
# ns_type -> ns-type
input_mdp_md = f"{MD_inp}md_charmm36m.mdp"

file=""";define                   =  -DPOSRES
;include                  = -I/home/villa/work/UseCaseI/forcefield
integrator               = md
tinit                    = 0
dt                       = 0.002
nsteps                   = 50000000   ; 100  ns
;nstcomm                  = 1
comm_grps                = system
nstxout                  = 0
nstvout                  = 0
nstfout                  = 0
nstlog                   = 100000
nstenergy                = 100000
nstxout-compressed       = 100000
compressed-x-grps        = system 
nstlist                  = 10
ns-type                  = grid
pbc                      = xyz
; neighbour
; rlist                   = 1.2 ; not used when cutoff-scheme = verlet
cutoff-scheme           = Verlet
; coulomb
coulombtype              = PME
rcoulomb                 = 1.2
fourierspacing           = 0.15 ;  For constant accuracy one should keep fourier-spacing/rcoulomb constant = 0.125
; vdw
vdwtype                  = Cut-off
vdw_modifier             = Force-switch
rvdw_switch              = 1.0  ; 
rvdw                     = 1.2
DispCorr                 = EnerPres ; while for lipid bilayer,  it's a difficult topic in the CHARMM force field. If one don't have lipids bi- or monolayers one should use it. 
;
constraints              = h-bonds
constraint_algorithm     = LINCS
; barostat
Pcoupl                   = Parrinello-Rahman   
tau_p                    = 5.0  
ref_p                    = 1.0 
compressibility          = 4.5e-5 
; thermostat
Tcoupl                   = v-rescale 
tc-grps                  = system
tau_t                    = 0.5 ; or 0.1 
ref_t                    = 298 """
with open(input_mdp_md, 'w') as f:
    f.write(file)

In [78]:
# Grompp: Creating portable binary run file for mdrun
from biobb_gromacs.gromacs.grompp import grompp

# Create prop dict and inputs/outputs

output_gppmd_tpr = f"{MD_dir}{pdbCode}_gppmd.tpr"

prop = {
    'mdp': {
        'nsteps' : 5_000_000, # 1 ns
        'nstxout' :  100_000
    },
    'gmx_lib' : f"{cwd}/{MD_inp}",
    'maxwarn' : 1
}

# Create and launch bb
grompp(input_gro_path=output_npt_gro, 
       input_top_zip_path=output_genion_top_zip, 
       input_mdp_path=input_mdp_md,
       output_tpr_path=output_gppmd_tpr, 
       input_cpt_path=output_npt_cpt, 
       properties=def_dict(prop))

2025-06-04 12:09:55,560 [MainThread  ] [INFO ]  Module: biobb_gromacs.gromacs.grompp Version: 5.0.0
2025-06-04 12:09:55,561 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_1ac5201f-249a-45a9-a746-c97b0531bd04 directory successfully created
2025-06-04 12:09:55,564 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K_npt.gro to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_1ac5201f-249a-45a9-a746-c97b0531bd04
2025-06-04 12:09:55,566 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K_npt.cpt to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_1ac5201f-249a-45a9-a746-c97b0531bd04
2025-06-04 12:09:55,567 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/input/md_charmm36m.mdp to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_1ac5201f-249a-45a9-a746-c97b0531bd04
2025-06-04 12:09:55,569 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/b

0

<a id="mdStep2"></a>
#### Step 2: Running short free MD simulation

In [79]:
# Mdrun: Running free dynamics
from biobb_gromacs.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_md_trr = f'{MD_dir}{pdbCode}_md.trr'
output_md_gro = f'{MD_dir}{pdbCode}_md.gro'
output_md_edr = f'{MD_dir}{pdbCode}_md.edr'
output_md_log = f'{MD_dir}{pdbCode}_md.log'
output_md_cpt = f'{MD_dir}{pdbCode}_md.cpt'

# Create and launch bb
# mdrun(input_tpr_path=output_gppmd_tpr, 
#       output_trr_path=output_md_trr, 
#       output_gro_path=output_md_gro, 
#       output_edr_path=output_md_edr, 
#       output_log_path=output_md_log, 
#       output_cpt_path=output_md_cpt,
#       properties=def_dict())

<a id="post"></a>
***
### Post-processing and Visualizing resulting 3D trajectory
Post-processing and Visualizing the **protein system** MD setup **resulting trajectory** using **NGL**
- [Step 1](#ppStep1): *Imaging* the resulting trajectory, **stripping out water molecules and ions** and **correcting periodicity issues**.
- [Step 2](#ppStep2): Generating a *dry* structure, **removing water molecules and ions** from the final snapshot of the MD setup pipeline.
- [Step 3](#ppStep3): Visualizing the *imaged* trajectory using the *dry* structure as a **topology**. 
***
**Building Blocks** used:
 - [GMXImage](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_image) from **biobb_analysis.gromacs.gmx_image** 
 - [GMXTrjConvStr](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_trjconv_str) from **biobb_analysis.gromacs.gmx_trjconv_str** 
***

<a id="ppStep1"></a>
#### Step 1: *Imaging* the resulting trajectory.
Stripping out **water molecules and ions** and **correcting periodicity issues**  

In [ ]:
# GMXImage: "Imaging" the resulting trajectory
#           Removing water molecules and ions from the resulting structure
from biobb_analysis.gromacs.gmx_image import gmx_image

# Create prop dict and inputs/outputs
output_imaged_traj = f'{MD_dir}{pdbCode}_imaged_traj.trr'
prop = {
    'center_selection':  'Protein',
    'output_selection': 'Protein',
    'pbc' : 'nojump',
}

# Create and launch bb
gmx_image(input_traj_path=output_md_trr,
          input_top_path=output_gppmin_tpr,
          output_traj_path=output_imaged_traj, 
          properties=def_dict(prop))

2025-06-04 12:05:02,914 [MainThread  ] [INFO ]  Module: biobb_analysis.gromacs.gmx_image Version: 5.0.1
2025-06-04 12:05:02,915 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_ef99ad73-5a3d-4b9e-8dd7-4a3b2daba5dc directory successfully created
2025-06-04 12:05:02,935 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/data/antibody/1_MD/4G6K_md.trr to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_ef99ad73-5a3d-4b9e-8dd7-4a3b2daba5dc
2025-06-04 12:05:02,939 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/data/antibody/1_MD/4G6K_gppmin.tpr to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_ef99ad73-5a3d-4b9e-8dd7-4a3b2daba5dc
2025-06-04 12:05:02,939 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/6ae0808c-5c89-4b30-ac0

0

In [72]:
# GMXImage: "Imaging" the resulting trajectory
#           Removing water molecules and ions from the resulting structure
from biobb_analysis.gromacs.gmx_image import gmx_image

# Create prop dict and inputs/outputs
output_imaged_traj_rot = f'{MD_dir}{pdbCode}_imaged_traj_rot.trr'
prop = {
    'center_selection':  'Protein',
    'output_selection': 'Protein',
    'fit' : 'rot+trans',
    'center' : True
}

# Create and launch bb
gmx_image(input_traj_path=output_imaged_traj,
          input_top_path=output_gppmd_tpr,
          output_traj_path=output_imaged_traj_rot, 
          properties=def_dict(prop))

2025-06-04 12:05:07,478 [MainThread  ] [INFO ]  Module: biobb_analysis.gromacs.gmx_image Version: 5.0.1
2025-06-04 12:05:07,479 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_e02c79e6-14e3-470c-9f00-3db6d149a801 directory successfully created
2025-06-04 12:05:07,481 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/data/antibody/1_MD/4G6K_imaged_traj.trr to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_e02c79e6-14e3-470c-9f00-3db6d149a801
2025-06-04 12:05:07,485 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/data/antibody/1_MD/4G6K_gppmd.tpr to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_e02c79e6-14e3-470c-9f00-3db6d149a801
2025-06-04 12:05:07,485 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/c284f75f-3211-

0

<a id="ppStep2"></a>
#### Step 2: Generating the output *dry* structure.
**Removing water molecules and ions** from the resulting structure

In [81]:
# GMXTrjConvStr: Converting and/or manipulating a structure
#                Removing water molecules and ions from the resulting structure
#                The "dry" structure will be used as a topology to visualize 
#                the "imaged dry" trajectory generated in the previous step.
from biobb_analysis.gromacs.gmx_trjconv_str import gmx_trjconv_str

# Create prop dict and inputs/outputs
output_dry_gro = f'{MD_dir}{pdbCode}_md_dry.gro'
prop = {
    'selection':  'Protein'
}

# Create and launch bb
gmx_trjconv_str(input_structure_path=output_md_gro,
                input_top_path=output_gppmd_tpr,
                output_str_path=output_dry_gro, 
                properties=def_dict(prop))

2025-06-04 12:11:03,331 [MainThread  ] [INFO ]  Module: biobb_analysis.gromacs.gmx_trjconv_str Version: 5.0.1
2025-06-04 12:11:03,332 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_cfe50716-6505-4f9d-9774-9f3a7c1fe42e directory successfully created
2025-06-04 12:11:03,335 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/data/antibody/1_MD/4G6K_md.gro to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_cfe50716-6505-4f9d-9774-9f3a7c1fe42e
2025-06-04 12:11:03,339 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/data/antibody/1_MD/4G6K_gppmd.tpr to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_cfe50716-6505-4f9d-9774-9f3a7c1fe42e
2025-06-04 12:11:03,340 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/2bb698e0-6095-4a9

0

<a id="ppStep3"></a>
#### Step 3: Visualizing the generated dehydrated trajectory.
Using the **imaged trajectory** (output of the [Post-processing step 1](#ppStep1)) with the **dry structure** (output of the [Post-processing step 2](#ppStep2)) as a topology.

In [82]:
# Show trajectory
view = nv.show_simpletraj(nv.SimpletrajTrajectory(output_imaged_traj_rot, output_dry_gro), gui=True)
view.add_licorice()
view

NGLWidget(max_frame=50)

<a id="mdStep3"></a>
#### Step 4: Checking free MD simulation results
Checking results for the final step of the setup process, the **free MD run**. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time during the **free MD run** step. **RMSd** against the **experimental structure** (input structure of the pipeline) and against the **minimized and equilibrated structure** (output structure of the NPT equilibration step).

In [36]:
# GMXRms: Computing Root Mean Square deviation to analyse structural stability 
#         RMSd against experimental structure (backbone atoms)   

from biobb_analysis.gromacs.gmx_rms import gmx_rms

# Create prop dict and inputs/outputs
output_rms_exp = f'{MD_dir}{pdbCode}_rms_exp.xvg'
prop = {
    'selection':  'Backbone',
    #'selection': 'non-Water'
}

# Create and launch bb
gmx_rms(input_structure_path=output_gppmin_tpr,
         input_traj_path=output_imaged_traj_rot,
         output_xvg_path=output_rms_exp, 
          properties=def_dict(prop))

2025-06-04 10:27:36,636 [MainThread  ] [INFO ]  Module: biobb_analysis.gromacs.gmx_rms Version: 5.0.1
2025-06-04 10:27:36,637 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_1d5c9980-989c-4255-ab26-f414f3aa1e86 directory successfully created
2025-06-04 10:27:36,639 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/data/antibody/1_MD/4G6K_gppmin.tpr to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_1d5c9980-989c-4255-ab26-f414f3aa1e86
2025-06-04 10:27:36,642 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/data/antibody/1_MD/4G6K_imaged_traj_rot.trr to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_1d5c9980-989c-4255-ab26-f414f3aa1e86
2025-06-04 10:27:36,642 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/99e7ef81-be

0

In [37]:
import plotly
import plotly.graph_objs as go

# Read RMS vs experimental structure data from file 
with open(output_rms_exp,'r') as rms_exp_file:
    x2,y2 = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_exp_file
            if not line.startswith(("#","@")) 
        ])
    )

trace = go.Scatter(
    x = x2,
    y = y2,
    name = 'RMSd vs exp'
)

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": trace,
    "layout": go.Layout(title="RMSd during free MD Simulation",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "RMSd (nm)")
                       )
}

plotly.offline.iplot(fig)


In [38]:
# GMXRgyr: Computing Radius of Gyration to measure the protein compactness during the free MD simulation 

from biobb_analysis.gromacs.gmx_rgyr import gmx_rgyr

# Create prop dict and inputs/outputs
output_rgyr = f'{MD_dir}{pdbCode}_rgyr.xvg'
prop = {
    'selection':  'Backbone'
}

# Create and launch bb
gmx_rgyr(input_structure_path=output_gppmin_tpr,
         input_traj_path=output_imaged_traj_rot,
         output_xvg_path=output_rgyr, 
          properties=def_dict(prop))

2025-06-04 10:27:36,806 [MainThread  ] [INFO ]  Module: biobb_analysis.gromacs.gmx_rgyr Version: 5.0.1
2025-06-04 10:27:36,807 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_b1993f73-be26-42e8-b3ff-1a67c5a92eb7 directory successfully created
2025-06-04 10:27:36,810 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/data/antibody/1_MD/4G6K_gppmin.tpr to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_b1993f73-be26-42e8-b3ff-1a67c5a92eb7
2025-06-04 10:27:36,814 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/data/antibody/1_MD/4G6K_imaged_traj_rot.trr to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_b1993f73-be26-42e8-b3ff-1a67c5a92eb7
2025-06-04 10:27:36,815 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/66c9e076-a

0

In [39]:
import plotly
import plotly.graph_objs as go

# Read Rgyr data from file 
with open(output_rgyr,'r') as rgyr_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rgyr_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Radius of Gyration",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "Rgyr (nm)")
                       )
}

plotly.offline.iplot(fig)

#### Step 5. Clustering CDR loops

In [41]:
fixed_pdb = f'{MD_dir}{pdbCode}_fixed.pdb'

In [47]:
!code {anarcii_pdb}

In [42]:
from anarcii import Anarcii

model = Anarcii(seq_type="antibody", mode="accuracy", verbose=True)
# results renunbers the file and returns the numbering
results = model.number(fixed_pdb)
anarcii_pdb = f'{MD_dir}4g6k_fixed-anarcii-imgt.pdb'
shutil.move('4g6k_fixed-anarcii-imgt.pdb', anarcii_pdb)

Using device CUDA with 16 CPUs
Batch size: 32
	Speed is a balance of batch size and length diversity. Adjust accordingly. For a full explanation see:
 	wiki/FAQs#recommended-batch-sizes
 	Seqs all similar length (+/-5), increase batch size. Mixed lengths (+/-30), reduce.


Consider larger batch size for optimal GPU performance.

Length of sequence list: 2
Processing sequences in 1 chunks of 102400 sequences.
Processing chunk 1 of 1.

 2 Long sequences detected - running in sliding window. This is slow.
Max probability windows selected.

Making predictions on 1 batches.
PDBx model index, chain ID: 0, H
ANARCII chain type (score): H (30.58661460876465)
 Sequence length: 128
 Sequence: [((1, ' '), 'Q'), ((2, ' '), 'V'), ((3, ' '), 'Q'), ((4, ' '), 'L'), ((5, ' '), 'Q'), ((6, ' '), 'E'), ((7, ' '), 'S'), ((8, ' '), 'G'), ((9, ' '), 'P'), ((10, ' '), '-'), ((11, ' '), 'G'), ((12, ' '), 'L'), ((13, ' '), 'V'), ((14, ' '), 'K'), ((15, ' '), 'P'), ((16, ' '), 'S'), ((17, ' '), 'Q'), ((18, ' ')

'data/antibody/1_MD/4g6k_fixed-anarcii-imgt.pdb'

In [43]:
from anarcii.output_data_processing.schemes_constants import schemes

# Show protein
view = nv.show_structure_file(anarcii_pdb)
# https://www.imgt.org/IMGTScientificChart/Nomenclature/IMGT-FRCDRdefinition.html
view.add_ball_and_stick(selection='27-38')   #CDR1
view.add_ball_and_stick(selection='56-65')   #CDR2
view.add_ball_and_stick(selection='105-117') #CDR3
view

NGLWidget()

In [48]:
from biobb_gromacs.gromacs.pdb2gmx import pdb2gmx
anarcii_gro = f'{MD_dir}4g6k_fixed-anarcii-imgt.gro'
anarcii_zip = f'{MD_dir}4g6k_fixed-anarcii-imgt.zip'

prop = {
    'force_field' : 'charmm36-jul2017',
    'gmx_lib' : f'{cwd}/{MD_inp}',
    'water_type' : 'tip3p',
    #'force_field' : 'amber99sb-ildn'
}

# Create and launch bb
pdb2gmx(input_pdb_path=anarcii_pdb, 
        output_gro_path=anarcii_gro, 
        output_top_zip_path=anarcii_zip,
        properties=def_dict(prop))

2025-06-04 18:46:56,384 [MainThread  ] [INFO ]  Module: biobb_gromacs.gromacs.pdb2gmx Version: 5.0.0
2025-06-04 18:46:56,385 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_fe7a7c17-e656-437d-9955-0270e4b6de2f directory successfully created
2025-06-04 18:46:56,386 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4g6k_fixed-anarcii-imgt.pdb to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_fe7a7c17-e656-437d-9955-0270e4b6de2f
2025-06-04 18:46:56,386 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright pdb2gmx -f /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_fe7a7c17-e656-437d-9955-0270e4b6de2f/4g6k_fixed-anarcii-imgt.pdb -o /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_fe7a7c17-e656-437d-9955-0270e4b6de2f/4g6k_fixed-anarcii-imgt.gro -p p2g.top -water tip3p -ff charmm36-jul2017 -i posre.itp

2025-06-04 18:46:57,003 [MainThread  ] [INFO ]  

0

In [54]:
from biobb_gromacs.gromacs.make_ndx import make_ndx

# Create prop dict and inputs/outputs
loop_ndx = f'{MD_dir}{pdbCode}_loop.ndx'
sele = 'r 27-38 | r 56-65 | r 105-117\nname 10 Loop'

prop = { 
    'selection': sele
}

# Create and launch bb
make_ndx(input_structure_path=anarcii_gro,
         output_ndx_path=loop_ndx,
         properties=def_dict(prop))

2025-06-04 18:48:21,082 [MainThread  ] [INFO ]  Module: biobb_gromacs.gromacs.make_ndx Version: 5.0.0
2025-06-04 18:48:21,083 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_a254e5bb-5152-4416-8194-7196599c6e76 directory successfully created
2025-06-04 18:48:21,084 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4g6k_fixed-anarcii-imgt.gro to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_a254e5bb-5152-4416-8194-7196599c6e76
2025-06-04 18:48:21,085 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/d9f89f2d-da54-4d9c-b175-5e1e984f96e5.stdin to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_a254e5bb-5152-4416-8194-7196599c6e76
2025-06-04 18:48:21,085 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright make_ndx -f /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_a254e5bb-5152-4416-8194-719659

0

In [56]:
output_gppmin_tpr = f'{MD_dir}{pdbCode}_gppmin.tpr'
output_imaged_traj_rot = f'{MD_dir}{pdbCode}_imaged_traj_rot.trr'

In [57]:
from biobb_analysis.gromacs.gmx_cluster import gmx_cluster

output_pdb_cluster = f'{MD_dir}{pdbCode}_clusters.pdb'

prop = {
    'fit_selection': 'Loop',
    'output_selection': 'System',
#    'method': 'gromos',
    'method' : 'jarvis-patrick',
    'cutoff' : 0.1,
    'nofit' : True,
}
gmx_cluster(input_structure_path=output_gppmin_tpr,
            input_traj_path=output_imaged_traj_rot,
            input_index_path=loop_ndx,
            output_pdb_path=output_pdb_cluster,
            properties=def_dict(prop))

2025-06-04 18:48:59,364 [MainThread  ] [INFO ]  Module: biobb_analysis.gromacs.gmx_cluster Version: 5.0.1
2025-06-04 18:48:59,366 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_3fc2b69f-693e-45e8-ab7e-9930dba8c3c8 directory successfully created
2025-06-04 18:48:59,370 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/data/antibody/1_MD/4G6K_gppmin.tpr to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_3fc2b69f-693e-45e8-ab7e-9930dba8c3c8
2025-06-04 18:48:59,375 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/data/antibody/1_MD/4G6K_imaged_traj_rot.trr to /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/sandbox_3fc2b69f-693e-45e8-ab7e-9930dba8c3c8
2025-06-04 18:48:59,376 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb/biobb_wf_antibody/biobb_wf_antibody/notebooks/data/an

0

In [58]:
import MDAnalysis as mda
u1  = mda.Universe(anarcii_pdb)
u2  = mda.Universe(output_pdb_cluster)

residsH = u1.select_atoms('chainID H').residues.resids
maskH = ((27 <= residsH) & (residsH <= 38)) | ((56 <= residsH) & (residsH <= 65)) | ((105 <= residsH) & (residsH <= 117))
residsA = u2.select_atoms('chainID A').residues.resids

residsL = u1.select_atoms('chainID L').residues.resids
maskL = ((27 <= residsL) & (residsL <= 38)) | ((56 <= residsL) & (residsL <= 65)) | ((105 <= residsL) & (residsL <= 117))
residsB = u2.select_atoms('chainID B').residues.resids

/home/rchaves/miniforge3/envs/biobb_wf_antibody/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:453: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"
/home/rchaves/miniforge3/envs/biobb_wf_antibody/lib/python3.10/site-packages/MDAnalysis/topology/PDBParser.py:331: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "


In [59]:
view = nv.show_structure_file(output_pdb_cluster, default_representation=False)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='all', color='modelindex')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view.add_ball_and_stick(selection=f"( {', '.join(list(map(str, residsA[maskH])))} ) and :A", color='modelindex')
view.add_ball_and_stick(selection=f"( {', '.join(list(map(str, residsB[maskL])))} ) and :B", color='modelindex')
view


NGLWidget()

## Docking the generated structures with HADDOCK

In [5]:
dock_dir = out_path+'2_docking/'

### Prepare PDBs

[pdb format](https://www.bonvinlab.org/haddock3-user-manual/structure_requirements.html#pdb-format)

#### Antibody

In [ ]:
!cat {output_pdb_cluster}  | pdb_reres -1 | pdb_chain -A | pdb_tidy -strict > {dock_dir}ab_clean.pdb

#### Antigen

In [254]:
!pdb_fetch 4I1B | pdb_tidy -strict | pdb_delhetatm | pdb_selaltloc | pdb_keepcoord | pdb_chain -B | pdb_chainxseg | pdb_tidy -strict > {dock_dir}ag_clean.pdb

#### Reference

In [253]:
!pdb_fetch 4G6M | pdb_tidy -strict | pdb_selchain -H | pdb_delhetatm | pdb_fixinsert | pdb_selaltloc | pdb_keepcoord | pdb_tidy -strict > {dock_dir}4G6M_H.pdb
!pdb_fetch 4G6M | pdb_tidy -strict | pdb_selchain -L | pdb_delhetatm | pdb_fixinsert | pdb_selaltloc | pdb_keepcoord | pdb_tidy -strict > {dock_dir}4G6M_L.pdb
!pdb_fetch 4G6M | pdb_tidy -strict | pdb_selchain -A | pdb_delhetatm | pdb_fixinsert | pdb_selaltloc | pdb_keepcoord | pdb_reres -1 | pdb_chain -B | pdb_chainxseg | pdb_tidy -strict > {dock_dir}4G6M_A.pdb
!pdb_merge {dock_dir}4G6M_H.pdb {dock_dir}4G6M_L.pdb | pdb_reres -1 | pdb_chain -A | pdb_chainxseg | pdb_tidy -strict > {dock_dir}4G6M_HL.pdb
!pdb_merge {dock_dir}4G6M_HL.pdb {dock_dir}4G6M_A.pdb | pdb_tidy -strict > {dock_dir}ref_clean.pdb

### Generate AIRs with epitope

In [292]:
!arctic3d-restraints --r1 {arc_path} --r2 {arc_path} --run_dir {dock_dir}arctic3d_res/

[2025-06-04 17:40:04,713 cli_restraints INFO] Starting arctic3d-restraints
[2025-06-04 17:40:04,740 output INFO] Creating output_directory data/antibody/2_docking/arctic3d_res
[2025-06-04 17:40:04,740] cli_restraints.py:arctic3d:main:260: Input files are {'r1_res_fname': PosixPath('data/antibody/2_docking/arctic3d/clustered_residues_probs.out'), 'r2_res_fname': PosixPath('data/antibody/2_docking/arctic3d/clustered_residues_probs.out')}
[2025-06-04 17:40:04,740] output.py:arctic3d.log:setup_output_folder:64: Setting up output folder data/antibody/2_docking/arctic3d_res
[2025-06-04 17:40:04,740] output.py:arctic3d.log:setup_output_folder:73: File clustered_residues_probs.out already exists, adding _1 to the filename
[2025-06-04 17:40:04,740] cli_restraints.py:arctic3d:main:268: data/antibody/2_docking/arctic3d/ residues = {1: [120, 121, 122, 127, 129, 130, 131, 135, 136, 137, 138, 139, 140, 141, 143, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 162, 164, 167, 168, 169, 170, 172

In [ ]:
!pdb_fetch 4I1B > {dock_dir}ag.pdb

In [9]:
# Obtain epitope
arc_path = 'data/antibody/2_docking/arctic3d/'
!arctic3d P01584 --db /home/rchaves/repo/arctic3d/db/swissprot --pdb_to_use 4I1B --run_dir {arc_path}

[2025-06-04 17:51:09,480 cli INFO] 
##############################################
#                                            #
#                 ARCTIC-3D                  #
#                                            #
##############################################

Starting ARCTIC-3D 0.4.1


Traceback (most recent call last):
  File "/home/rchaves/miniforge3/envs/biobb_wf_antibody/bin/arctic3d", line 8, in <module>
    sys.exit(maincli())
  File "/home/rchaves/miniforge3/envs/biobb_wf_antibody/lib/python3.10/site-packages/arctic3d/cli.py", line 169, in maincli
    return cli(argument_parser, main)
  File "/home/rchaves/miniforge3/envs/biobb_wf_antibody/lib/python3.10/site-packages/arctic3d/cli.py", line 164, in cli
    return main_func(**vars(cmd))
  File "/home/rchaves/miniforge3/envs/biobb_wf_antibody/lib/python3.10/site-packages/arctic3d/cli.py", line 213, in main
    run_dir_path = create_output_folder(run_dir, uniprot_id)
  File "/home/rchaves/miniforge3/envs/biobb_wf_antibo

In [ ]:
epitopes = !cat {arc_path}clustered_residues.out

# Convert space-separated string to list of integers
residue_numbers1 = list(map(int, epitopes[0].split('>')[1].split()))

[119,
 120,
 121,
 122,
 125,
 127,
 129,
 130,
 131,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 143,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 157,
 162,
 164,
 167,
 168,
 169,
 170,
 171,
 172,
 180,
 181,
 182,
 183,
 188,
 189,
 190,
 191,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 208,
 209,
 210,
 212,
 213,
 214,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 231,
 232,
 233,
 234,
 242,
 243,
 244,
 245,
 246,
 247,
 256,
 257,
 263,
 264,
 265,
 266,
 267,
 268,
 269]

In [280]:
# Convert to Uniprot numbering
!pdb_fetch 4I1B | pdb_reres -119 > {dock_dir}ag_uni.pdb

In [62]:
view = nv.show_structure_file(f'{dock_dir}ag_uni.pdb')
for cluster, color in zip(epitopes, ['red', 'blue', 'green', 'orange', 'purple']):
    residue_numbers = list(map(int, cluster.split('>')[1].split()))
    res_sel = f"{', '.join(map(str, residue_numbers))}"
    print(res_sel)
    view.add_ball_and_stick(selection=res_sel, color=color, radius=0.2)
#view.add_surface(selection='not HOH',color='residueindex')
view

119, 120, 121, 122, 125, 127, 129, 130, 131, 135, 136, 137, 138, 139, 140, 141, 143, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 162, 164, 167, 168, 169, 170, 171, 172, 180, 181, 182, 183, 188, 189, 190, 191, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 208, 209, 210, 212, 213, 214, 219, 220, 221, 222, 223, 224, 225, 231, 232, 233, 234, 242, 243, 244, 245, 246, 247, 256, 257, 263, 264, 265, 266, 267, 268, 269
119, 120, 121, 122, 123, 159, 177, 178, 179, 182, 183, 184, 203, 204, 206, 207, 269
130, 170, 220, 222, 223, 225, 227, 242, 253, 254, 255, 256, 257, 258, 260, 261


NGLWidget()

In [34]:
residue_numbers

[119,
 120,
 121,
 122,
 125,
 127,
 129,
 130,
 131,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 143,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 157,
 162,
 164,
 167,
 168,
 169,
 170,
 171,
 172,
 180,
 181,
 182,
 183,
 188,
 189,
 190,
 191,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 208,
 209,
 210,
 212,
 213,
 214,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 231,
 232,
 233,
 234,
 242,
 243,
 244,
 245,
 246,
 247,
 256,
 257,
 263,
 264,
 265,
 266,
 267,
 268,
 269]

In [ ]:
view = nv.show_structure_file(f'{dock_dir}ref_clean.pdb')
# view.clear()
residue_numbers = list(map(int, cluster.split('>')[1].split()))
res_sel = f"{', '.join(map(str, residue_numbers))}"
print(res_sel)
view.add_ball_and_stick(selection=res_sel, color=color, radius=0.2)
view

NGLWidget()

### Run HADDOCK3

In [ ]:
!haddock3 data/antibody/2_docking/docking-antibody-antigen.cfg

## PMX

In [ ]:
# https://mmb.irbbarcelona.org/biobb/workflows/tutorials/biobb_wf_pmx_tutorial
# https://github.com/bioexcel/biobb_wf_pmx_tutorial

***
<a id="questions"></a>

## Questions & Comments

Questions, issues, suggestions and comments are really welcome!

* GitHub issues:
    * [https://github.com/bioexcel/biobb](https://github.com/bioexcel/biobb)

* BioExcel forum:
    * [https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library](https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library)
